In [0]:
-- For whatever reason, part1 and part2 when ran together drives the performance super slow.
-- So the code for part1 is commented.

-- Part2 was done using A LOT cursor. My simplex knowledge from the previous millenium has vanished.
-- I could re-code the algo myself but there are so many little stuff here and there I'm not confident to proceed.
-- Still got correct result in 20-30 seconds on a 2XS, I'll take that!


with cte_data (machine_id, tgt_lights, buttons_list, joltage) as
(
select monotonically_increasing_id() + 1
     , translate(regexp_extract(value, r'^\[(.+)\]'), '.#', '01')
     , transform(regexp_extract_all(value, r'\(([^)]+)\)', 1), t -> array_sort(transform(split(t, ','), u -> u::int)))
     , transform(split(regexp_extract(value, r'\{(.+)\}$'), ','), t -> t::int)
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_10_input.txt', format => 'text', wholeText => false) as t (value)
/*from values ('[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}')
            , ('[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}')
            , ('[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}') as t (value) */
)
  ,  cte_prep (machine_id, nb_indic, tgt_b10, buttons_list, buttons_b10, inc, joltage, ai) as
(
select machine_id
     , length(tgt_lights)
     , conv(tgt_lights, 2, 10)::int
     , buttons_list
     , transform(buttons_list, t -> aggregate(t, 0, (acc, x) -> acc + (1 << (length(tgt_lights) - x - 1))))
     , transform(buttons_list, b -> aggregate(b, 0, (acc, x) -> acc + power(10, length(tgt_lights) - x - 1)::int, acc -> transform(split(lpad(acc, length(tgt_lights), '0'), ''), u -> u::int))) as inc
     , joltage
     , transform(inc, t -> array_min(filter(transform(arrays_zip(joltage, t), u -> u.joltage * u.t), f -> f > 0))) as ai
  from cte_data
 where true
-- and machine_id = 1
)
/*,  cte_part1 as
(
select sum(aggregate( sequence(1,10)
                    , struct(0 as n, array(0) as s)
                    , (acc, n) -> case array_size(filter(acc.s, f -> f = tgt_b10))
                                    when 0
                                    then struct(n, array_distinct(flatten(transform(acc.s, a -> transform(buttons_b10, b -> a ^ b)))) as s)
                                    else acc
                                  end
                    , acc -> acc.n
                    ) ) as part1
  from cte_prep
) */
  ,  cte_base as
(
select machine_id
     , joltage as cible
     , inc
  from cte_prep
),
cte_matrix AS (
  SELECT 
    machine_id,
    transform(sequence(0, size(cible) - 1), r -> 
      cast(array_append(transform(sequence(0, size(inc) - 1), c -> inc[c][r]), cible[r]) as array<double>)
    ) as mat,
    size(cible) as num_rows,
    size(inc) as num_vars,
    array_max(cible) as max_val
  FROM cte_base
),

cte_forward AS (
  SELECT machine_id,
    reduce(sequence(0, least(num_rows, num_vars) - 1), mat,
      (m, k) -> transform(m, (row, i) ->
        CASE
          WHEN i < k THEN row
          WHEN i = k THEN m[coalesce(filter(sequence(k, size(m)-1), j -> m[j][k] != 0)[0], k)]
          ELSE
            CASE
              WHEN i = coalesce(filter(sequence(k, size(m)-1), j -> m[j][k] != 0)[0], k) THEN m[k]
              WHEN m[coalesce(filter(sequence(k, size(m)-1), j -> m[j][k] != 0)[0], k)][k] = 0 THEN row
              ELSE transform(row, (v, c) -> 
                v - (row[k] / m[coalesce(filter(sequence(k, size(m)-1), j -> m[j][k] != 0)[0], k)][k]) 
                  * m[coalesce(filter(sequence(k, size(m)-1), j -> m[j][k] != 0)[0], k)][c])
            END
        END
      )
    ) as ref, 
    num_rows, num_vars, max_val
  FROM cte_matrix
),

cte_with_fnz AS (
  SELECT machine_id, ref, num_rows, num_vars, max_val,
    transform(ref, row -> coalesce(filter(sequence(0, num_vars - 1), c -> abs(row[c]) > 0.0001d)[0], -1)) as fnz
  FROM cte_forward
),

cte_fix AS (
  SELECT machine_id, num_rows, num_vars, max_val,
    reduce(sequence(0, least(num_rows, 9) - 1), ref,
      (m, p) -> transform(m, (row, r) ->
        CASE 
          WHEN r <= p OR fnz[r] < 0 OR fnz[r] != fnz[p] OR abs(m[p][fnz[p]]) < 0.0001d THEN row
          ELSE transform(row, (v, c) -> v - (row[fnz[p]] / m[p][fnz[p]]) * m[p][c])
        END
      )
    ) as ref
  FROM cte_with_fnz
),

cte_pivots AS (
  SELECT machine_id, ref, num_rows, num_vars, max_val,
    transform(sequence(0, size(ref) - 1), r -> coalesce(filter(sequence(0, num_vars - 1), c -> abs(ref[r][c]) > 0.0001d)[0], -1)) as pcr
  FROM cte_fix
),

cte_rref AS (
  SELECT machine_id, num_vars, max_val,
    transform(sequence(0, size(pcr) - 1), r ->
      CASE WHEN pcr[r] < 0 OR r != coalesce(filter(sequence(0, size(pcr) - 1), i -> pcr[i] = pcr[r])[0], -1) THEN -1 ELSE pcr[r] END
    ) as pivot_cols,
    filter(sequence(0, size(pcr) - 1), r -> pcr[r] >= 0 AND r = coalesce(filter(sequence(0, size(pcr) - 1), i -> pcr[i] = pcr[r])[0], -1)) as valid_rows,
    reduce(
      filter(sequence(0, size(pcr) - 1), r -> pcr[r] >= 0 AND r = coalesce(filter(sequence(0, size(pcr) - 1), i -> pcr[i] = pcr[r])[0], -1)),
      ref,
      (m, r) -> transform(m, (row, i) ->
        CASE WHEN i = r THEN transform(row, (v, c) -> CASE WHEN abs(m[r][pcr[r]]) < 0.0001d THEN v ELSE v / m[r][pcr[r]] END)
          ELSE transform(row, (v, c) -> CASE WHEN abs(m[r][pcr[r]]) < 0.0001d THEN v ELSE v - (row[pcr[r]] / m[r][pcr[r]]) * m[r][c] END)
        END)
    ) as rref
  FROM cte_pivots
),

cte_bounds AS (
  SELECT machine_id, rref, num_vars, max_val, pivot_cols, valid_rows,
    filter(sequence(0, num_vars - 1), c -> NOT array_contains(filter(pivot_cols, p -> p >= 0), c)) as free_vars,
    transform(
      filter(sequence(0, num_vars - 1), c -> NOT array_contains(filter(pivot_cols, p -> p >= 0), c)),
      fv -> named_struct(
        'u', cast(floor(aggregate(sequence(0, size(rref) - 1), cast(max_val as double),
          (ub, r) -> CASE WHEN coalesce(rref[r][fv], 0d) > 0.01d AND coalesce(rref[r][num_vars], 0d) > 0d 
            AND size(filter(filter(sequence(0, num_vars - 1), c -> NOT array_contains(filter(pivot_cols, p -> p >= 0), c)), 
                            fv2 -> fv2 != fv AND coalesce(rref[r][fv2], 0d) < -0.01d)) = 0
            THEN least(ub, rref[r][num_vars] / rref[r][fv]) ELSE ub END)) as int),
        'l', cast(ceiling(aggregate(sequence(0, size(rref) - 1), 0d,
          (lb, r) -> CASE WHEN coalesce(rref[r][num_vars], 0d) < 0d AND coalesce(rref[r][fv], 0d) < -0.01d
            AND size(filter(filter(sequence(0, num_vars - 1), c -> NOT array_contains(filter(pivot_cols, p -> p >= 0), c)),
                            fv2 -> coalesce(rref[r][fv2], 0d) < -0.01d)) = 1
            THEN greatest(lb, (-rref[r][num_vars]) / (-rref[r][fv])) ELSE lb END)) as int)
      )
    ) as bounds
  FROM cte_rref
),

cte_ranges AS (
  SELECT *, size(free_vars) as nf,
    CASE WHEN size(bounds) >= 1 AND bounds[0].l <= bounds[0].u THEN sequence(bounds[0].l, bounds[0].u) ELSE array(0) END as r0,
    CASE WHEN size(bounds) >= 2 AND bounds[1].l <= bounds[1].u THEN sequence(bounds[1].l, bounds[1].u) ELSE array(0) END as r1,
    CASE WHEN size(bounds) >= 3 AND bounds[2].l <= bounds[2].u THEN sequence(bounds[2].l, bounds[2].u) ELSE array(0) END as r2,
    CASE WHEN size(bounds) >= 4 AND bounds[3].l <= bounds[3].u THEN sequence(bounds[3].l, bounds[3].u) ELSE array(0) END as r3
  FROM cte_bounds
)
,
cte_combos AS
(
SELECT machine_id, rref, num_vars, free_vars, pivot_cols, valid_rows, cast(array() as array<int>) as combo FROM cte_ranges WHERE nf = 0
 UNION ALL
SELECT machine_id, rref, num_vars, free_vars, pivot_cols, valid_rows, array(v0) FROM cte_ranges, LATERAL explode(r0) t0(v0) WHERE nf = 1
 UNION ALL
SELECT machine_id, rref, num_vars, free_vars, pivot_cols, valid_rows, array(v0,v1) FROM cte_ranges, LATERAL explode(r0) t0(v0), LATERAL explode(r1) t1(v1) WHERE nf = 2
 UNION ALL
SELECT machine_id, rref, num_vars, free_vars, pivot_cols, valid_rows, array(v0,v1,v2) FROM cte_ranges, LATERAL explode(r0) t0(v0), LATERAL explode(r1) t1(v1), LATERAL explode(r2) t2(v2) WHERE nf = 3
 UNION ALL
SELECT machine_id, rref, num_vars, free_vars, pivot_cols, valid_rows, array(v0,v1,v2,v3) FROM cte_ranges, LATERAL explode(r0) t0(v0), LATERAL explode(r1) t1(v1), LATERAL explode(r2) t2(v2), LATERAL explode(r3) t3(v3) WHERE nf = 4
) 
,
cte_solved AS (
  SELECT machine_id,
    reduce(reverse(valid_rows),
      transform(sequence(0, num_vars - 1), var -> CASE WHEN array_position(free_vars, var) > 0 THEN cast(combo[array_position(free_vars, var) - 1] as double) ELSE -9999d END),
      (sol, r) -> transform(sol, (v, var) ->
        CASE WHEN var != pivot_cols[r] THEN v
          ELSE coalesce(rref[r][num_vars], 0d) - aggregate(sequence(0, num_vars - 1), 0d,
            (s, c) -> CASE WHEN c = var OR sol[c] < -9000d THEN s ELSE s + coalesce(rref[r][c], 0d) * sol[c] END)
        END)
    ) as sol
  FROM cte_combos
),
cte_valid AS (
  SELECT machine_id, 
    transform(sol, v -> cast(round(v) as int)) as solution,
    aggregate(sol, 0, (a, v) -> a + cast(round(v) as int)) as total_cost
  FROM cte_solved
  WHERE aggregate(sol, true, (ok, v) -> ok AND v >= -0.01d AND v < 9000d AND abs(v - round(v)) < 0.1d)
qualify ROW_NUMBER() OVER (PARTITION BY machine_id ORDER BY total_cost) = 1
)
  ,  cte_part2 as
(
select sum(total_cost) as part2
  from cte_valid
)
select part2 from cte_part2
;
